In [3]:
import numpy as np
# --- Metrics ---
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, classification_report

acc = accuracy_score(y_test, y_pred)

# Always return a 2x2 confusion matrix for interpretability (even if a class is missing)
cm = confusion_matrix(y_test, y_pred, labels=[0, 1])

# AUC only makes sense if both classes appear in y_test
try:
    auc = roc_auc_score(y_test, y_cont)
except ValueError:
    auc = float("nan")

# Build a classification report that adapts to present classes
labels_present = np.unique(np.concatenate([y_test, y_pred])).astype(int)
name_map = {0: f"Not {pos_label}", 1: pos_label}
target_names_present = [name_map[l] for l in labels_present]

rep = classification_report(
    y_test,
    y_pred,
    labels=labels_present,
    target_names=target_names_present,
    digits=4,
    zero_division=0
)

# --- Threshold sweep (optional) ---
ts = np.linspace(0, 1, 201)
accs = [((y_cont >= t).astype(int) == y_test).mean() for t in ts]
best_idx = int(np.argmax(accs))
best_t, best_acc = float(ts[best_idx]), float(accs[best_idx])

# --- Save ---
metrics = {
    "threshold_default": 0.5,
    "accuracy_default": float(acc),
    "roc_auc_continuous": float(auc),
    "confusion_matrix_labels": ["0=Not_"+pos_label, "1="+pos_label],
    "confusion_matrix": cm.tolist(),
    "classification_report": rep,
    "present_class_labels": labels_present.tolist(),
    "present_class_names": target_names_present,
    "best_threshold": best_t,
    "best_accuracy": best_acc,
}
with open(EVAL_JSON, "w") as f:
    json.dump(metrics, f, indent=2)

print("=== Evaluation ===")
if len(labels_present) < 2:
    print("NOTE: Only one class is present in y_test (or predictions). Some metrics are undefined.")
print(f"Default threshold 0.5 | Accuracy: {acc:.4f} | ROC AUC: {auc:.4f}")
print("Confusion matrix [[TN, FP],[FN, TP]]:\n", cm)
print("\nClassification report:\n", rep)
print(f"Best threshold: {best_t:.3f} | Best accuracy: {best_acc:.4f}")


=== Evaluation ===
NOTE: Only one class is present in y_test (or predictions). Some metrics are undefined.
Default threshold 0.5 | Accuracy: 1.0000 | ROC AUC: nan
Confusion matrix [[TN, FP],[FN, TP]]:
 [[ 0  0]
 [ 0 10]]

Classification report:
               precision    recall  f1-score   support

  Republican     1.0000    1.0000    1.0000        10

    accuracy                         1.0000        10
   macro avg     1.0000    1.0000    1.0000        10
weighted avg     1.0000    1.0000    1.0000        10

Best threshold: 0.000 | Best accuracy: 1.0000


/home/aimlexpert/~/venvs/arrow/bin/python /lib/python3.12/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
